In [2]:
import tensorflow as tf
import numpy as np

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
minist = input_data.read_data_sets('./dataset/minist/', one_hot=True)

Extracting ./dataset/minist/train-images-idx3-ubyte.gz
Extracting ./dataset/minist/train-labels-idx1-ubyte.gz
Extracting ./dataset/minist/t10k-images-idx3-ubyte.gz
Extracting ./dataset/minist/t10k-labels-idx1-ubyte.gz


In [4]:
print(minist.train.images.shape)
print(minist.train.labels.shape)
minist.train.num_examples

(55000, 784)
(55000, 10)


55000

In [5]:
print(minist.test.images.shape)
print(minist.test.labels.shape)
minist.test.num_examples

(10000, 784)
(10000, 10)


10000

In [6]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

In [7]:
XR = tf.reshape(X, [-1, 28, 28, 1])

In [9]:
W1 = tf.Variable(tf.random_normal([3,3,1,60], stddev=0.01))
W2 = tf.Variable(tf.random_normal([3,3,60,120], stddev=0.01))
W3 = tf.Variable(tf.random_normal([3,3,120,240], stddev=0.01))
# W1

In [10]:
L1 = tf.nn.relu(
    tf.nn.conv2d(XR, W1, strides=[1,1,1,1], padding='SAME')
)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L1 = tf.nn.dropout(L1, 0.8)

L2 = tf.nn.relu(
    tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L2 = tf.nn.dropout(L2, 0.8)

L3 = tf.nn.relu(
    tf.nn.conv2d(L2, W3, strides=[1,1,1,1], padding='SAME')
)
L3 = tf.nn.max_pool(L3, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
L3 = tf.nn.dropout(L3, 0.8)
L3

<tf.Tensor 'dropout_2/mul:0' shape=(?, 4, 4, 240) dtype=float32>

In [9]:
# FC
FCL = tf.reshape(L3, [-1, 4*4*240])
print(FCL)
# FCW = tf.Variable(tf.random_normal([4*4*240, 10]))
FCW = tf.get_variable('FCW', shape=[4*4*240, 10], initializer=tf.contrib.layers.xavier_initializer())

FCB = tf.Variable(tf.random_normal([10]))
print(FCW)
H = tf.matmul(FCL, FCW)+FCB
C = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits=H, labels=Y)
)

Tensor("Reshape_1:0", shape=(?, 3840), dtype=float32)
Tensor("FCW/read:0", shape=(3840, 10), dtype=float32)


In [10]:
# T = tf.train.AdamOptimizer(0.1).minimize(C)
T = tf.train.RMSPropOptimizer(0.1, 0.9).minimize(C)

In [12]:
# batch_size = 5000
batch_size = 100
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(5):
        avg_cost = 0

        total_batch = int(minist.train.num_examples / batch_size)
        print(total_batch)
        
        for i in range(total_batch):
            trainx, trainy = minist.train.next_batch(100)
            _, c = sess.run([T, C], feed_dict={X: trainx, Y: trainy})
            
            print(c)

            avg_cost += c / total_batch
            
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.3f}'.format(avg_cost))

#     a = sess.run(A, feed_dict={X: minist.test.images, Y: minist.test.labels, KB: 1})
#     print('{0} %'.format(a))

550
2.86543
2.88298
2.76971
2.80618
2.88788
2.81031
2.62702
2.55778
2.76676
2.75677
2.66898
2.73102
2.72595
2.70165
2.65245
2.57171
2.59265
2.53546
2.57796
2.47985
2.68082
2.60562
2.5057
2.52047
2.41266
2.48928
2.27355
2.37098
2.85467
2.50917
2.48244
2.36935
2.45967
2.52339
2.43627
2.42983
2.52162
2.32003
2.34557
2.41473
2.36689
2.36478
2.36624
2.30278
2.31946
2.35095
2.36726
2.29841
2.32492
2.31772
2.35876
2.33356
2.33459
2.32061
2.29555
2.30735
2.30081
2.28795
2.30527
2.32134
2.31696
2.29492
2.29964
2.29966
2.29755
2.30578
2.28804
2.30674
2.29696
2.29894
2.31539
2.23637
5.98546
2.3755
2.30053
2.97139
2.48342
2.47141
20.5678
2.4104
2.31418
2.33403
2.33618
2.30841
2.29096
3.32755
2.33298
2.32523
2.34141
2.30876
2.31256
2.30288
2.30503
2.30691
2.30224
2.31469


KeyboardInterrupt: 